In [ ]:
import pandas as pd
file_path = '/home/yuhaowang/project/FMBC/downstream/finetune/dataset_csv/others'
csv_path = 'GTEX_BreastGENE.csv'
data = pd.read_csv(csv_path)
print(len(data.columns))
data.head()

5484


,silde_id,TSPAN6,TNMD,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,GCLC,...,TOX2,SRSF6,GDAP1L1,GTSF1L,ARFGEF2,ZNFX1,ZNF831,EDN3,CSE1L,TMEM189-UBE2V1
0,GTEX-ZA64-1526,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.822439,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,GTEX-X4EP-2926,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.758760,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,GTEX-S7SE-0826,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,GTEX-XMD1-0826,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,GTEX-11P81-192,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.814639,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
import pandas as pd
import os
file_path = '/home/yuhaowang/project/FMBC/downstream/finetune/dataset_csv/others'
for file in os.listdir(file_path):
    print(file)
    data = pd.read_csv(os.path.join(file_path, file))
    print(len(data.columns))

Multi-omic_RNASEQ.csv
57909
TCGA-BRCA_GENECOUNTS.csv
59428
TIGER-TIL-SCORES-WSITILS.csv
2
TCGA-BRCA_FPKMS_LOG2.csv
59428
BreakHis_MALIGNANT_4SUBTYPE.csv
2
CPTAC_GENE.csv
8344
TCGA-BRCA_TPMS_LOG2.csv
59428
GTEX_BreastGENE.csv
5484
BreakHis_BENIGN_4SUBTYPE.csv
2


/tmp/ipykernel_1049146/3998511732.py:6: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(os.path.join(file_path, file))
